# Steam Games Analysis

In [6]:
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd

load_dotenv()

conn = pymysql.connect(host='localhost',
                        port=int(3306),
                        user='root',
                        passwd=os.getenv('MYSQL_PASSWORD'),
                        db='steam')

user_rnkd_games = pd.read_sql_query('''
                    WITH steamid_tot_playtime AS (
                    SELECT
                        steamid,
                        SUM(playtime_forever) AS tot_playtime_forever
                    FROM
                        games_daily gd
                    WHERE
                        DATE(dateretrieved) = '2014-11-09'
                    GROUP BY
                        steamid)
                    SELECT
                        gd.steamid,
                        appid,
                        playtime_forever,
                        tot_playtime_forever,
                        COALESCE(playtime_forever / NULLIF(tot_playtime_forever,0),0) AS rating,
                        ROW_NUMBER() OVER(PARTITION BY steamid ORDER BY rating DESC) steamid_appid_rnk
                    FROM
                        games_daily gd
                    LEFT JOIN steamid_tot_playtime  ON
                        steamid_tot_playtime.steamid = gd.steamid 
                    WHERE
                        DATE(dateretrieved) = '2014-11-09'                  
                    ''', conn)

games = pd.read_sql_query('''
                    SELECT ai.appid, Title, Genre, Price, Is_Multiplayer
                    FROM app_id_info ai
                    LEFT JOIN games_genres
                    ON ai.appid = games_genres.appid
                    ''', conn)



In [7]:
user_rnkd_games[user_rnkd_games['steamid_appid_rnk'] ==  1].merge(games, how='left')

,steamid,appid,playtime_forever,tot_playtime_forever,rating,steamid_appid_rnk,Title,Genre,Price,Is_Multiplayer
0,76561197960267727,550,17270,64110.0,0.2694,1,Left 4 Dead 2,Action,19.99,1.0
1,76561197960268178,550,11252,31458.0,0.3577,1,Left 4 Dead 2,Action,19.99,1.0
2,76561197960268875,730,55508,70047.0,0.7924,1,Counter-Strike: Global Offensive,Action,14.99,1.0
3,76561197960269081,570,16910,34539.0,0.4896,1,Dota 2,Action,0.00,1.0
4,76561197960269081,570,16910,34539.0,0.4896,1,Dota 2,Free to Play,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...
32717,76561197987333972,240,7972,7972.0,1.0000,1,Counter-Strike: Source,Action,19.99,1.0
32718,76561197987345232,300,50761,50873.0,0.9978,1,Day of Defeat: Source,Action,9.99,1.0
32719,76561197987348222,10180,677,794.0,0.8526,1,Call of Duty®: Modern Warfare® 2,Action,19.99,1.0
32720,76561197987348632,240,26708,97872.0,0.2729,1,Counter-Strike: Source,Action,19.99,1.0
